### Closure
+ Thread(target=f)
+ model.compile(loss=f)
+ QPushButton(connect=f)

In [24]:
from threading import Thread

# no args
def hello():
    print("hello")
t = Thread(target=hello)
t.start()
t.join()

# args: way 1
def wrap(name):
    def hello2():
        print(f"hello {name}")
    return hello2
t = Thread(target=wrap("James"))
t.start()
t.join()

# args: way 2
def hello2(name):
    print(f"hello {name}")
t = Thread(target=lambda :hello2("James"))
t.start()
t.join()

hello
hello James
hello James


### Decorator: add syntax without modify function and calling code
+ It's not important that whether the raw function has input args or not since we won't modify calling code
1. Do something before function
    + e.g. logging

In [14]:
def printx1(x):
    print(x)
    
def printx2(x):
    print(x**2)
    
printx1(3)
printx2(3)

3
9


In [21]:
import logging

# way0: must modify calling code so not good
def wrap(f):
    logging.warning(f"{f.__name__} is running")
    return f

def printx1(x):
    print(x)
    
def printx2(x):
    print(x**2)
    
wrap(printx1)(3)
wrap(printx2)(3)

3
9


In [22]:
# way1: wrap is a "function in function out" function and can be decorator syntax
def wrap(f):
    logging.warning(f"{f.__name__} is running")
    return f

@wrap
def printx1(x):
    print(x)
    
@wrap
def printx2(x):
    print(x**2)
    
printx1(3)
printx2(3)

3
9


2. do something before and after function
    + e.g. timer

In [29]:
import time

def wrap(f):
    def execute(name):
        s = time.time()
        f(name)
        print(time.time()-s) 
    return execute

@wrap
def hello(name):
    print(name)
    
hello("James")

James
0.0004119873046875


3. do something general
    + Closure on decorator
    + Remember we always use "def wrap(f) -> f" for decorators
    + e.g. Repeat n times

In [37]:
def repeat(n):
    def wrap(f):
        def execute(name):
            for i in range(n):
                s = time.time()
                f(name)
                print(time.time()-s)
        return execute
    return wrap

@repeat(3)
def hello(name):
    print(name)
    
hello("James")

James
0.000308990478515625
James
5.245208740234375e-06
James
1.9073486328125e-06


### Example practice: Cache

In [54]:
D = {}
def repeat(n):
    def wrap(f):
        def execute(x):
            for i in range(n):
                s = time.time()
                if x in D:
                    result = D[x]
                else:
                    result = f(x)
                    D[x] = result
                print(f"time={time.time()-s}")
            return result
        return execute
    return wrap

@repeat(5)
def accum(n):
    return sum( i for i in range(int(n+1)) ) 

print( accum(1e6) )

time=0.07469391822814941
time=9.5367431640625e-07
time=0.0
time=0.0
time=9.5367431640625e-07
500000500000


### Functool cache

In [59]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fac(n):
    return n*fac(n-1) if n else 1

for i in range(3):
    s = time.time()
    print( fac(5) )
    print(time.time()-s)

120
0.0003139972686767578
120
5.9604644775390625e-06
120
3.0994415283203125e-06
